In [1]:
import numpy as np
from itertools import permutations
from copy import deepcopy

In [2]:
### Hepler functions ###
def fill_area(start_row, start_col, target, recipe, multiplier):
    """ Hepler function for filling out subarea of target matrix 
        with content of recipe * multiplier
    Args:
        start_row : Integer = The beginning row index of area
        start_col : Integer = The beginning col index of area
        target    : 2D numpy array to be filled in subarea
        recipe    : 2D numpy array that goes into subarea of target
        multiplier: Float (possibly complex) multiplied onto all recipe vals
    """
    row_counter = 0
    for row in range(start_row,start_row + 2):
        col_counter = 0
        for col in range(start_col, start_col + 2):
            target[row][col] = recipe[row_counter][col_counter] * multiplier
            col_counter += 1
        row_counter+=1

def recursive_perm_scoring(mat, perms, test_mat, row_idx):
    """Computing all possibles alignment scores of MSA matrix; mat,
       given the permutations for each of the N-1 rows of mat, (found in perms)
    
    Args:
        mat     : 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                              ['A', 'C', '_', '_'],
                                              ['A', 'T', '_', '_']])
        perms   : all distinct permutations of the N-1 last rows of mat
        test_mat: matrix 
       """
    perm_history, score_history = [], []
    if row_idx < mat.shape[0]:
        for i in range(0 , len(perms[row_idx - 1])):
            test_mat[row_idx,:] = perms[row_idx - 1][i]
            score_list, perm_list = recursive_perm_scoring(mat, perms, test_mat, row_idx + 1)
            perm_history += perm_list
            score_history += score_list
            current_score = 0
            for j in range(0 , mat.shape[1]):
                current_score += score_sequence(test_mat[:,j])
            score_history.append(current_score)
            perm_history.append(deepcopy(test_mat))
    return score_history, perm_history

In [3]:
### Main functions ###

def score(n1,n2):
    """ Function for computing the score of
        some alignment of the two chars n1, n2
    Args:
        n1: any str in {"_","A","T","C","G"}
        n2: any str in {"_","A","T","C","G"}
    Returns:
        score: int in {-1,0,1}
    """
    score = None
    gap = "_"
    if   n1 == gap or n2 == gap: score =  0; return score
    elif n1 == n2              : score = -1; return score
    elif n1 != n2              : score =  1; return score

def encode_score_weights(mat):
    """Encoding the score of all possible alignments
        for all n1, n2 for all s1 < s2 score(n1,n2)

    Args:
        mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                         ['A', 'C', '_', '_'],
                                         ['A', 'T', '_', '_']])   
    Returns:
        weight_matrices: 3D numpy array of shape (1/2 * (L - 1) * L , C , C)
                         where L = nr. of rows and C = nr. of cols in
                         matrix given
    """
    L, C = mat.shape
    weight_matrices = [np.zeros((C,C)) for i in range(int(1/2 * (L - 1) * L))]
    for row1 in range(0 , mat.shape[0]):
        for row2 in range(row1 + 1 , mat.shape[0]):
            for idx1, n1 in enumerate(mat[row1,:]):
                for idx2, n2 in enumerate(mat[row2,:]):
                    weight_matrices[row1+row2-1][idx1][idx2] = score(n1,n2)
    return np.array(weight_matrices)

def matrix_2_bit_state(mat):
    """
    Maps some given matrix repr. of a MSA to a corresponding
    bitstring via the column encoding: x_(s,n,i) determines whether
    the n'th letter of the s'th string is placed in the i'th column.

    Args:
        mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                         ['A', 'C', '_', '_'],
                                         ['A', 'T', '_', '_']])
    Returns:
        numpy array containing bit repr., e.g.: np.array([1,0,0,1...])
    """
    regs = []
    gap = "_"
    for row in range(my_mat.shape[0]):           # For each S
        for col in range(my_mat.shape[1]):       # For each n
            if my_mat[row][col] != gap:
                current_reg = [] 
                for i in range(my_mat.shape[0]+1):     # for each i
                    if i == col: current_reg.append(1)
                    else:        current_reg.append(0)
                regs.append(current_reg)
    return np.array(regs).flatten()

def bit_state_2_matrix(bit_string,init_mat):
    """
    Maps some given bitstring repr. of a MSA to a corresponding
    matrix via the initial matrix. 

    Args:
        bit_string: numpy array containing bit repr., e.g.: np.array([1,0,0,1...])
        mat       : 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                                ['A', 'C', '_', '_'],
                                                ['A', 'T', '_', '_']])
    Returns:
        2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                    ['A', 'C', '_', '_'],
                                    ['A', 'T', '_', '_']])
    """
    counts, gap = [], "_"
    for row in range(init_mat.shape[0]):
        current_count = 0
        for col in range(init_mat.shape[1]):
            if init_mat[row][col] != gap: current_count += 1
        counts.append(current_count)

    lower = 0
    multiplier, regs = init_mat.shape[1], []
    for value in counts:
        for i in range(value):
            regs.append(bit_string[lower + i * multiplier : lower + (i + 1) * multiplier])
        lower += value * multiplier

    counter = 0
    new_mat = np.zeros((init_mat.shape),dtype=object)
    for row in range(init_mat.shape[0]):
        for col in range(init_mat.shape[1]):
            if init_mat[row][col] != gap:
                col_index = np.where(regs[counter] == 1)[0][0]
                new_mat[row][col_index] = init_mat[row][col]
                counter += 1
            else: new_mat[row][col] = gap

    return new_mat


def tensor_state(str):
    """ Creates a numpy vector representation of a tensor
    product state consisting of |0> 's and |1> 's

    Args:
        string of zeros and ones, e.g. '10'

    Returns:
        np.array([0,1,0,0,...])
    """
    zero, one = np.array([1,0]), np.array([0,1])
    basis_objects = [zero,one]
    objects = []
    objects.append(basis_objects[int(str[-1])])

    for idx in range(0 , len(str) - 1):
        current_object = np.zeros(2**(idx+2))
        counter = 0
        for i in range(2):
            for j in range(len(objects[-1])):
                current_object[counter] = objects[-1][j] * basis_objects[int(str[idx])][i]
                counter += 1
        objects.append(current_object)
    return objects[-1]


def multiple_matrix_tensor(string):
    """ Creates series of tensor product of square matrix repr.
    of multiple pauli operators 

    Args:
        string: A string representation of the pauli operators
                where X = Pauli_X, Y = Pauli_Y, Z = Pauli_Z, I = identity
    
    Returns:
        2D numpy array corresponding to matrix repr. of tensor product
    """
    complex_vals = False
    for str in string:
        if str == "Y": complex_vals = True

    sigma_x, sigma_y, sigma_z = np.array([[0,1],[1,0]]),np.array([[0,-1j],[1j,0]]),np.array([[1,0],[0,-1]])
    identity = np.identity(2)
    basis_map = {"X":sigma_x,"Y":sigma_y,"Z":sigma_z,"I":identity}

    objects = []
    objects.append(basis_map[string[0]])
    for idx in range(0 , len(string) - 1):
        if complex_vals:
            current_object = np.zeros((2 ** (idx+2) , 2 ** (idx+2)),dtype=complex)
        else:
            current_object = np.zeros((2 ** (idx+2) , 2 ** (idx+2)))
        row_counter = 0
        for i in range(int(current_object.shape[0] / 2)):
            col_counter = 0
            for j in range(int(current_object.shape[0] / 2)):
                if objects[-1][i][j] == 1:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],1)
                elif objects[-1][i][j] == -1:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],-1)
                elif objects[-1][i][j] == 1j:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],1j)
                elif objects[-1][i][j] == -1j:
                    fill_area(row_counter,col_counter,current_object,basis_map[string[idx+1]],-1j)
                col_counter += 2
            row_counter += 2                
        objects.append(current_object)
    return objects[-1]


def initial_MSA_matrix(strings):
    """ Creating a matrix representation of the strings given
    and filling gaps with "_"

    Args:
        list of strings, e.g. ["ACCT","AC","AT"]

    Returns:
        2D numpy array
    """
    lengths = [len(str) for str in strings]
    initial_matrix = np.zeros((len(strings) , np.max(lengths)),dtype=object)
    for row in range(initial_matrix.shape[0]):
        for col in range(len(strings[row])):
            initial_matrix[row][col] = strings[row][col]
    initial_matrix[initial_matrix == 0] = "_"
    return initial_matrix

def MSA_binary_matrix(matrix):
    """ Mapping matrix of alphabet strings to 
    corresponding binary strings 
    
    Args:
        2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                    ['A', 'C', '_', '_'],
                                    ['A', 'T', '_', '_']])
    Returns:
        Corresponding mapped 2D numpy array
    """
    alphabet_dict = {"_":'000',"A":'001',"C":'010',"T":'011',"G":'100'}
    resulting_matrix = np.zeros(matrix.shape,dtype=object)
    for row in range(matrix.shape[0]):
        for col in range(matrix.shape[1]):
            binary = alphabet_dict[matrix[row][col]]
            resulting_matrix[row][col] = binary
    return resulting_matrix

def hamming_distance(state1,state2):
    """Calculates the hamming distance between 
    state1 and state2 

    Args:
        state1: string repr. of tensor state, e.g. "001" or "111" or ...
        state2: string repr. of tensor state, e.g. "001" or "111" or ...
    Returns:
        dist: integer score equivalent to number of differing characters
    """
    dist = 0
    for i in range(0 , len(state1)):
        if state1[i] != state2[i]: dist += 1
    return dist 


def legal_permutations(arr): 
    """ Function for computing all permutations of array
    of type ["A","C","_","T","_"] that maintains original
    order of characters != "_" .

    Args:
        arr: numpy array, e.g.: np.array(["A","C","_","T","_"])
    Returns:
        2D numpy array of legal permutations, e.g.: np.array([["A","C","_","T","_"],
                                                              ["A","C","_","_","T"],...])

    """
    legal_perm_indices = []
    letter_order = [char for char in arr if char != "_"]
    perms = list(set(permutations(arr))) # Using set to remove dubs
    perms = [list(perm) for perm in perms]
    for idx, perm in enumerate(perms):
        letter_counter = 0
        keep_perm = True
        for letter in perm:
            if letter != "_":
                if letter_order[letter_counter] != letter:
                    keep_perm = False
                else: letter_counter += 1
        if keep_perm: legal_perm_indices.append(idx)
    return np.array(perms)[legal_perm_indices]


def score_sequence(arr):
    """Function for computing sum-of-pairs score
       according to scheme defined in 
       score function. 
    Args:
        arr: numpy array, e.g.: np.array(["A","C","_","T","_"])
    Returns:
        final_score: integer 
       """
    final_score = 0
    for n1 in range(0 , len(arr)):
        for n2 in range(n1 + 1 , len(arr)):
            final_score += score(arr[n1],arr[n2])
    return final_score

def fixed_brute_force(init_mat):
    """ Initial version of brute force scoring of all relevant
    permutations of MSA matrix (Only works for 3xN sized matrix).
    Make sure the longest row containin  only letters are initially 
    placed at top of matrix.
    
    Args:
        Init_mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                              ['A', 'C', '_', '_'],
                                              ['A', 'T', '_', '_']])
    Returns:
        best_score: Integer
        best perms: list of corresponding permutations of MSA matrix
    """
    perms = []
    for row_idx in range(1 , init_mat.shape[0]):
        perms.append(legal_permutations(init_mat[row_idx,:]))
    test_mat = np.zeros((init_mat.shape),dtype=object)
    test_mat[0,:] = init_mat[0,:]
    mat_history   = []
    score_history = []
    for row_idx_1 in range(1 , test_mat.shape[0]):
        for perm_1 in range(0 , len(perms[row_idx_1 - 1])):
            test_mat[row_idx_1,:] = perms[row_idx_1 - 1][perm_1]
            for row_idx_2 in range(row_idx_1 + 1 , test_mat.shape[0]):
                for perm_2 in range(0 , len(perms[row_idx_2 - 1])):
                    test_mat[row_idx_2,:] = perms[row_idx_2 - 1][perm_2]
                    current_perm_score = 0
                    for col in range(0 , test_mat.shape[1]):
                        current_perm_score += score_sequence(test_mat[:,col])
                    mat_history.append(deepcopy(test_mat))
                    score_history.append(current_perm_score)
    best_score = np.min(score_history)
    best_perms = [mat_history[i] for i in range(len(mat_history)) if score_history[i] == best_score]
    return best_score, best_perms



    
def recursive_brute_force(init_mat):
    """ Final recursive version of brute force scoring of all relevant
    permutations of variable sized MSA matrix. 
    (Make sure the longest row containin only letters are
     initially placed at top of matrix)
     
    Args:
        Init_mat: 2D numpy array, e.g. array([['A', 'C', 'C', 'T'],
                                              ['A', 'C', '_', '_'],
                                              ['A', 'T', '_', '_']])
    Returns:
        best_score: Integer
        best perms: list of corresponding permutations of MSA matrix
    """
    perms = np.array([list(set(permutations(init_mat[i]))) for i in range(1,len(init_mat))])
    test_mat = np.zeros((init_mat.shape) , dtype=object)
    test_mat[0,:] = init_mat[0,:]
    score_history, mat_history = recursive_perm_scoring(init_mat,perms,test_mat,1)
    best_score = np.min(score_history)
    best_perms = [mat_history[i] for i in range(len(mat_history)) if score_history[i] == best_score]
    return best_score, best_perms



    

In [4]:
initial_strings = ["ACCT","AC","AT"]
my_mat = initial_MSA_matrix(initial_strings)
my_mat

array([['A', 'C', 'C', 'T'],
       ['A', 'C', '_', '_'],
       ['A', 'T', '_', '_']], dtype=object)

In [5]:
fixed_brute_force(my_mat)

(-5,
 [array([['A', 'C', 'C', 'T'],
         ['A', '_', 'C', '_'],
         ['A', '_', '_', 'T']], dtype=object),
  array([['A', 'C', 'C', 'T'],
         ['A', 'C', '_', '_'],
         ['A', '_', '_', 'T']], dtype=object)])

In [6]:
recursive_brute_force(my_mat)

(-5,
 [array([['A', 'C', 'C', 'T'],
         ['A', '_', 'C', '_'],
         ['A', '_', '_', 'T']], dtype=object),
  array([['A', 'C', 'C', 'T'],
         ['A', 'C', '_', '_'],
         ['A', '_', '_', 'T']], dtype=object)])

In [7]:
binary_repr = MSA_binary_matrix(my_mat)
binary_repr

array([['001', '010', '010', '011'],
       ['001', '010', '000', '000'],
       ['001', '011', '000', '000']], dtype=object)

In [8]:
some_state = binary_repr[0][0]
some_state

'001'

In [9]:
tensor_state(some_state)

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [10]:
multiple_matrix_tensor("ZZX")

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [11]:
initial_strings = ["ACCT","AC","AT"]
my_mat = initial_MSA_matrix(initial_strings)
my_mat

array([['A', 'C', 'C', 'T'],
       ['A', 'C', '_', '_'],
       ['A', 'T', '_', '_']], dtype=object)

In [12]:
init_mat = my_mat
init_mat

array([['A', 'C', 'C', 'T'],
       ['A', 'C', '_', '_'],
       ['A', 'T', '_', '_']], dtype=object)

In [13]:
bit_string = matrix_2_bit_state(init_mat)
bit_string

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0])

In [14]:
new_mat = bit_state_2_matrix(bit_string,init_mat)
new_mat

array([['A', 'C', 'C', 'T'],
       ['A', 'C', '_', '_'],
       ['A', 'T', '_', '_']], dtype=object)

In [15]:
encode_score_weights(new_mat)

array([[[-1.,  1.,  0.,  0.],
        [ 1., -1.,  0.,  0.],
        [ 1., -1.,  0.,  0.],
        [ 1.,  1.,  0.,  0.]],

       [[-1.,  1.,  0.,  0.],
        [ 1.,  1.,  0.,  0.],
        [ 1.,  1.,  0.,  0.],
        [ 1., -1.,  0.,  0.]],

       [[-1.,  1.,  0.,  0.],
        [ 1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]]])